In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = mls
band = [5]
commitnumber = de5d43e
molecule = {'h2o': 'atmpro'}
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band3c
  atmpro = mls
  band = 3c
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 5e-25
  molecule = h2o
  ng_adju = [0]
  ng_refs = [5]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 800
  vmin = 720
  w_diffuse = [(1.55, 1.6, 1.66, 1.66, 1.8)]
  wgt = [(0.5, 0.5, 0.6, 0.7, 0.9)]


# Cooling Rate by Layer

CRD               CLIRAD `coolr_bands`          
            layer        coolrg                layer    coolrg
pressure                                                      
0.000312      1.0 -7.332137e-06                  NaN       NaN
0.000312      NaN           NaN                  1.0 -0.000013
0.000750      2.0 -6.714662e-06                  2.0 -0.000013
0.001052      3.0 -5.351271e-06                  3.0 -0.000012
0.001476      4.0 -3.845447e-06                  4.0 -0.000012
0.002070      5.0 -2.269908e-06                  5.0 -0.000011
0.002904      6.0 -6.299917e-07                  6.0 -0.000011
0.004074      7.0  1.077172e-06                  7.0 -0.000010
0.005714      8.0  2.855360e-06                  8.0 -0.000010
0.008015      9.0  4.706660e-06                  9.0 -0.000009
0.011243     10.0  6.632064e-06                 10.0 -0.000008
0.015771     11.0  8.637204e-06                 11.0 -0.000008
0.022122     12.0  1.071992e-05                 12.0 -0.000007
0.031031     13.0  1.288702e-05                 13.0 -0.000006
0.043528     14.0  1.528539e-05                 14.0 -0.000005
0.061057     15.0  2.289568e-05                 15.0 -0.000003
0.085645     16.0  4.808131e-05                 16.0  0.000006
0.120136     17.0  9.844485e-05                 17.0  0.000025
0.168516     18.0  1.739997e-04                 18.0  0.000053
0.236378     19.0  2.830424e-04                 19.0  0.000093
0.331549     20.0  4.375364e-04                 20.0  0.000151
0.465100     21.0  6.528289e-04                 21.0  0.000231
0.652400     22.0  9.458306e-04                 22.0  0.000349
0.915100     23.0  1.245885e-03                 23.0  0.000470
1.283650     24.0  1.269348e-03                 24.0  0.000479
1.800600     25.0  1.022228e-03                 25.0  0.000385
2.525700     26.0  7.620041e-04                 26.0  0.000287
3.542800     27.0  5.506584e-04                 27.0  0.000206
4.969550     28.0  3.902550e-04                 28.0  0.000145
6.970850     29.0  2.744161e-04                 29.0  0.000100
9.778100     30.0  1.907055e-04                 30.0  0.000068
13.715850    31.0  1.273329e-04                 31.0  0.000042
19.239350    32.0  8.000926e-05                 32.0  0.000023
26.987250    33.0  4.954378e-05                 33.0  0.000010
37.855300    34.0  3.079623e-05                 34.0  0.000002
53.100050    35.0  1.604577e-05                 35.0 -0.000005
73.887500    36.0  4.160113e-06                 36.0 -0.000010
97.662500    37.0 -1.988900e-06                 37.0 -0.000014
121.437500   38.0 -4.260249e-06                 38.0 -0.000015
145.212500   39.0 -6.223905e-06                 39.0 -0.000016
168.987500   40.0 -6.059462e-06                 40.0 -0.000017
192.762500   41.0  4.943874e-06                 41.0 -0.000017
216.537500   42.0  9.424351e-05                 42.0  0.000004
240.312500   43.0  4.353080e-04                 43.0  0.000165
264.087500   44.0  1.309218e-03                 44.0  0.000686
287.862500   45.0  3.046075e-03                 45.0  0.001915
311.637500   46.0  5.078867e-03                 46.0  0.003653
335.412500   47.0  7.449703e-03                 47.0  0.005877
359.187500   48.0  1.039904e-02                 48.0  0.008796
382.962500   49.0  1.392995e-02                 49.0  0.012416
406.737500   50.0  1.801749e-02                 50.0  0.016588
430.512500   51.0  2.261848e-02                 51.0  0.021116
454.287500   52.0  2.717921e-02                 52.0  0.025499
478.062500   53.0  3.194901e-02                 53.0  0.029963
501.837500   54.0  3.721105e-02                 54.0  0.034852
525.612500   55.0  4.300520e-02                 55.0  0.040193
549.387500   56.0  4.937696e-02                 56.0  0.046402
573.162500   57.0  5.643169e-02                 57.0  0.053610
596.937500   58.0  6.696224e-02                 58.0  0.064465
620.712500   59.0  7.832457e-02                 59.0  0.076322
644.487500   60.0  8.987480e-02              

# Fluxes by Layer

CRD                              CLIRAD             \
                         flug          fldg      fnetg       flug       fldg   
pressure     level                                                             
0.000000e+00 1     -30.744163  0.000000e+00 -30.744163        NaN        NaN   
1.000000e-08 1            NaN           NaN        NaN -30.806082   0.000001   
6.244000e-04 2     -30.744163  4.166112e-09 -30.744163 -30.806082   0.000001   
8.759000e-04 3     -30.744163  5.865661e-09 -30.744163 -30.806082   0.000001   
1.228600e-03 4     -30.744163  8.314810e-09 -30.744163 -30.806082   0.000001   
1.723400e-03 5     -30.744163  1.185102e-08 -30.744163 -30.806082   0.000001   
2.417400e-03 6     -30.744163  1.695572e-08 -30.744163 -30.806082   0.000001   
3.390900e-03 7     -30.744163  2.432492e-08 -30.744163 -30.806082   0.000001   
4.756500e-03 8     -30.744163  3.496248e-08 -30.744163 -30.806082   0.000001   
6.672000e-03 9     -30.744163  5.031598e-08 -30.744163 -30.806082   0.000001   
9.358900e-03 10    -30.744163  7.247545e-08 -30.744163 -30.806082   0.000001   
1.312780e-02 11    -30.744163  1.044554e-07 -30.744163 -30.806082   0.000001   
1.841450e-02 12    -30.744163  1.506057e-07 -30.744163 -30.806082   0.000001   
2.583020e-02 13    -30.744163  2.172011e-07 -30.744163 -30.806082   0.000001   
3.623230e-02 14    -30.744164  3.132954e-07 -30.744163 -30.806082   0.000001   
5.082340e-02 15    -30.744164  4.522025e-07 -30.744163 -30.806082   0.000001   
7.129060e-02 16    -30.744164  6.649804e-07 -30.744163 -30.806082   0.000001   
1.000000e-01 17    -30.744164  1.041610e-06 -30.744163 -30.806082   0.000002   
1.402710e-01 18    -30.744164  1.771039e-06 -30.744162 -30.806083   0.000002   
1.967600e-01 19    -30.744165  3.184764e-06 -30.744161 -30.806083   0.000003   
2.759970e-01 20    -30.744165  5.909233e-06 -30.744159 -30.806083   0.000004   
3.871000e-01 21    -30.744164  1.112799e-05 -30.744153 -30.806083   0.000006   
5.431000e-01 22    -30.744162  2.107803e-05 -30.744141 -30.806083   0.000010   
7.617000e-01 23    -30.744156  3.983688e-05 -30.744116 -30.806082   0.000018   
1.068500e+00 24    -30.744144  7.286107e-05 -30.744071 -30.806078   0.000031   
1.498800e+00 25    -30.744126  1.197493e-04 -30.744006 -30.806073   0.000051   
2.102400e+00 26    -30.744108  1.743675e-04 -30.743933 -30.806069   0.000074   
2.949000e+00 27    -30.744091  2.343081e-04 -30.743857 -30.806066   0.000100   
4.136600e+00 28    -30.744078  2.990095e-04 -30.743779 -30.806066   0.000128   
5.802500e+00 29    -30.744071  3.686502e-04 -30.743702 -30.806069   0.000160   
8.139200e+00 30    -30.744071  4.443581e-04 -30.743626 -30.806077   0.000196   
1.141700e+01 31    -30.744080  5.276806e-04 -30.743552 -30.806091   0.000237   
1.601470e+01 32    -30.744103  6.195412e-04 -30.743483 -30.806114   0.000283   
2.246400e+01 33    -30.744143  7.212829e-04 -30.743422 -30.806150   0.000336   
3.151050e+01 34    -30.744208  8.388495e-04 -30.743369 -30.806203   0.000400   
4.420010e+01 35    -30.744305  9.821127e-04 -30.743323 -30.806281   0.000481   
6.200000e+01 36    -30.744448  1.159300e-03 -30.743289 -30.806395   0.000586   
8.577500e+01 37    -30.744648  1.370926e-03 -30.743277 -30.806556   0.000718   
1.095500e+02 38    -30.744855  1.572664e-03 -30.743283 -30.806726   0.000848   
1.333250e+02 39    -30.745070  1.775061e-03 -30.743295 -30.806904   0.000983   
1.571000e+02 40    -30.745291  1.978658e-03 -30.743312 -30.807087   0.001121   
1.808750e+02 41    -30.745528  2.199076e-03 -30.743329 -30.807286   0.001273   
2.046500e+02 42    -30.745906  2.590521e-03 -30.743315 -30.807605   0.001543   
2.284250e+02 43    -30.746939  3.889312e-03 -30.743050 -30.808506   0.002456   
2.522000e+02 44    -30.749307  7.483471e-03 -30.741824 -30.810628   0.005042   
2.759750e+02 45    -30.754030  1.589331e-02 -30.738137 -30.814951   0.011298   
2.997500e+02 46    -30.762480  3.292240e-02 -30.729558 -30.822805   0.024547   
3.235250e+02 47    -30.774

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')